In [ ]:
import duckdb
import pyarrow.parquet as pq
import pandas as pd

In [ ]:
reviews = "../reviews.parquet"

pRev = pq.ParquetFile(reviews)
revSchema = pRev.schema_arrow
print(revSchema)


recommendationid: int64
appid: int64
author_steamid: int64
author_playtime_at_review: int64
author_last_played: int64
language: string
timestamp_created: timestamp[ns]
votes_up: int64
votes_funny: int64
weighted_vote_score: double
comment_count: int64
steam_purchase: int64
received_for_free: int64
written_during_early_access: int64
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1876


In [28]:
games = "../games.parquet"
pGames = pq.ParquetFile(games)
gamesSchema = pGames.schema_arrow
print(gamesSchema)

appid: int64
game: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 290


In [29]:
con = duckdb.connect()


query = f"""
    SELECT *
    FROM '{reviews}' AS r
    JOIN '{games}' AS g
    ON r.appid = g.appid
    LIMIT 10
"""

first10= con.execute(query).fetchdf()

print(first10)


   recommendationid   appid     author_steamid  author_playtime_at_review  \
0          61858151  105450  76561198095427016                         74   
1          61858130  105450  76561198367517870                       5972   
2          61857258  105450  76561198202501564                      15604   
3          61855517  105450  76561199014804619                        783   
4          61855022  105450  76561198147562649                         42   
5          61854906  105450  76561198274390926                      67134   
6          61851893  105450  76561198858100402                       1829   
7          61848535  105450  76561198818482403                      11231   
8          61844535  105450  76561198008998762                       1604   
9          61838163  105450  76561198207761347                        307   

   author_last_played    language   timestamp_created  votes_up  votes_funny  \
0          1590676756     english 2020-01-14 20:39:20         0         

In [53]:
countr = f"""
    SELECT COUNT(distinct appid) AS total_reviews
    FROM '{reviews}'
"""
df_reviews_count = con.execute(countr).fetchdf()
print(df_reviews_count["total_reviews"][0])

103167


In [31]:
countg = f"""
    SELECT COUNT(*) AS unique_games
    FROM '{games}'
"""
game_apid = con.execute(countg).fetchdf()
print(game_apid["unique_games"][0])

105893


In [32]:
countg = f"""
    SELECT COUNT(DISTINCT game) AS unique_games
    FROM '{games}'
"""
games_count = con.execute(countg).fetchdf()
print(games_count["unique_games"][0])

105206


In [33]:
dupes = f"""
    SELECT
        game,
        COUNT(DISTINCT appid) AS distinct_appids
    FROM '{games}'
    GROUP BY game
    HAVING COUNT(DISTINCT appid) > 1
    ORDER BY distinct_appids DESC
    LIMIT 10;
"""
qdupes = con.execute(dupes).fetchdf()
print(qdupes)

             game  distinct_appids
0            None               68
1           Alone                6
2          Aurora                6
3          Escape                6
4            Lost                4
5   Hide and Seek                4
6         Requiem                4
7        Parasite                4
8          Bounce                4
9  Space Survival                4


In [44]:
parq = "rev.parquet"

In [45]:
revp = pq.ParquetFile(parq).schema_arrow
print(revp)

recommendationid: int64
game: string
author_steamid: int64
author_playtime_at_review: int64
author_last_played: int64
language: string
timestamp_created: timestamp[ns]
votes_up: int64
votes_funny: int64
weighted_vote_score: double
comment_count: int64
steam_purchase: int64
received_for_free: int64
written_during_early_access: int64
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1877


In [51]:
pquery = f"""
    SELECT *
    FROM '{parq}'
    LIMIT 10
"""

f10=con.execute(pquery).fetchdf()

print(f10)

   recommendationid            game     author_steamid  \
0         148919893  Counter-Strike  76561199036724879   
1         148919350  Counter-Strike  76561198826729322   
2         148913051  Counter-Strike  76561199101237625   
3         148912714  Counter-Strike  76561198347582422   
4         148912575  Counter-Strike  76561198363716821   
5         148906148  Counter-Strike  76561198198293481   
6         148905699  Counter-Strike  76561198820122182   
7         148900627  Counter-Strike  76561198346217916   
8         148899121  Counter-Strike  76561199143791757   
9         148898785  Counter-Strike  76561198853145449   

   author_playtime_at_review  author_last_played  language  \
0                        197          1698336369   russian   
1                        441          1698335809   russian   
2                       1313          1698338635   turkish   
3                       1612          1698341834   russian   
4                        197          1698329401   

In [52]:
ugames = f"""
    SELECT COUNT(DISTINCT game) AS unique_games
    FROM '{parq}'
"""
unique_games = con.execute(ugames).fetchdf()
print(unique_games["unique_games"][0])

102513


In [60]:


query = f"""
    SELECT COUNT(*) FILTER (WHERE game IS NULL) AS missing_game,
    COUNT(*) FILTER (WHERE game = '') AS empty_game
    FROM '{parq}'
"""

res = con.execute(query).fetchdf()
print(res)


   missing_game  empty_game
0          5266           0


In [62]:


query = f"""
    SELECT *
    FROM '{parq}'
    WHERE game IS NULL
    LIMIT 10
"""

res = con.execute(query).fetchdf()
print(res)


   recommendationid  game     author_steamid  author_playtime_at_review  \
0          84555529  None  76561197971905574                        103   
1          66663227  None  76561198047671566                         72   
2          64124201  None  76561197970265034                        121   
3          56306807  None  76561197994407206                         68   
4          55664796  None  76561197974522156                         23   
5          53814851  None  76561198338012436                        171   
6          51837192  None  76561198149884953                         80   
7          51522225  None  76561198031466665                        259   
8          51019577  None  76561198122973075                        300   
9          50907952  None  76561198966624917                        122   

   author_last_played language   timestamp_created  votes_up  votes_funny  \
0          1610462658  english 2021-01-12 15:04:47         1            0   
1          158604023

In [67]:
out = "noNull.parquet"

con = duckdb.connect()

con.execute(f"CREATE TABLE reviews AS SELECT * FROM '{parq}'")

con.execute("""
    CREATE TABLE noNull AS
    SELECT *
    FROM reviews
    WHERE game IS NOT NULL
""")

con.execute(f"COPY noNull TO '{out}' (FORMAT PARQUET)")

row_count = con.execute("SELECT COUNT(*) FROM noNull").fetchone()[0]
print(f"Rows after removing NULL games: {row_count}")
print(f"Saved cleaned data to: {out}")

Rows after removing NULL games: 111378451
Saved cleaned data to: noNull.parquet


In [68]:
query = f"""
    SELECT COUNT(*) FILTER (WHERE game IS NULL) AS missing_game,
    COUNT(*) FILTER (WHERE game = '') AS empty_game
    FROM '{out}'
"""

res = con.execute(query).fetchdf()
print(res)


   missing_game  empty_game
0             0           0
